# Tabla Y4-125

In [ ]:
#!pip install specdal spectrum

In [ ]:
import os
from specdal import Collection, Spectrum, read
from matplotlib import pyplot as plt
import pandas as pd

datadir = "./Espectros_FORS_2/Y4"
archivos = list()
specs = list()
cont=0
aux = 0
etiquetas = ['A1','A2','A3','A4','A5',
             'B1','B2','B3','B4','B5',
             'C1','C2','C3','C4','C5',
             'D1','D2','D3','D4','D5',
             'E1','E2','E3','E4','E5',
            ]
for f in os.listdir(datadir):
    data = pd.read_csv(datadir+"/"+f,delimiter='\t')
    #print(f)
    archivos.append(f)
    dev_x = data['Wavelength']
    title =str(f)[:-4]
    dev_y = data[title]
    plt.plot(dev_x, dev_y)
    df = pd.DataFrame({'x': dev_x, 'y': dev_y})
    tmp_arr = df.to_numpy()
    specs.append(tmp_arr)
    aux+=1
    if aux == 5:
        aux=0
        plt.ylabel('Reflectance')
        plt.xlabel('Wavelength')
        t = "Tabla Y4",etiquetas[cont]
        cont+=1
        plt.title(t)
        plt.show()

#print(archivos)
len(specs)
print(cont)